# 08_analysis_PCA_HLA

## PCA and HLA Matching analysis

Initial analysis of bone-marrow data

#### Load libraries and data

In [ ]:
#| message: false
#| warning: false

library(tidyverse)
library(broom)
library(ggrepel)
library(ggridges)
library(patchwork)
library(here)

bm_data <- read_tsv(file = here("data/03_dat_aug.tsv.gz"), 
                    show_col_types = FALSE, na = "1000000")  |>
  mutate(across(c(1:2, 4:22, 24:26, 35), factor)) |>
  mutate(across(c(3,23,27:34), as.integer)) |>
  select(- time_to_aGvHD_III_IV) |>
  drop_na(where(is.numeric))

The time to GvGD column has been dropped as only a few number of the
patients develop the disease. Thus, to ensure that the PCA analysis is
based on as many observations as possible, the variable is left out.

In [ ]:
pca_fit <- bm_data |>
  select(where(is.numeric)) |>
  drop_na() |>
  prcomp(scale = TRUE)

r_matrix <- pca_fit |>
  tidy(matrix = "rotation")|>
  pivot_wider(names_from = "PC", names_prefix = "PC", values_from = "value")

pca_plot <- pca_fit |>
  augment(bm_data) |>
  ggplot(aes(x = .fittedPC1, 
             y = .fittedPC2, 
             colour = survival_status)) +
  geom_point(size = 1.5) +
  labs(x = "PC1 (31,2%)",
       y = "PC2 (15,5%)", 
       colour = "Survival Status",
       title = "Scores Plot") +
  scale_colour_viridis_d() +
  theme_minimal() +
  theme(legend.position = "bottom") 

pca_plot

The PC-scores plot shows that there is no clear separation between the
samples based on the survival status and thus, the groups are not
distinguishable based on the first two principal components.

In [ ]:
# define arrow style for plotting
arrow_style <- arrow(
  angle = 20, 
  ends = "first", 
  type = "closed", 
  length = grid::unit(8, "pt"))

# plot rotation matrix
rotation_plot <- pca_fit |>
  tidy(matrix = "rotation")|>
  pivot_wider(names_from = "PC", 
              names_prefix = "PC", 
              values_from = "value")|>
  ggplot(mapping = aes(PC1, PC2)) +
  geom_segment(xend = 0, 
               yend = 0, 
               arrow = arrow_style) +
  geom_text_repel(aes(label = column),
                  size = 3,
                  colour = "#440154FF",
                  hjust = 1,
                  nudge_x = 0.2) +
  labs(title = "Loading Plot") +
  scale_x_continuous(limits = c(-0.8,1.0)) +
  scale_y_continuous(limits = c(-0.8,1.0)) +
  theme_minimal() 

rotation_plot

The loading plot shows a strong correlation between the CD34+ and CD3
dosage and the first principal component (PC1). Whereas recipient age
and body mass are mainly correlated to PC1, but are also somewhat
correlated with the second principal component (PC2).  
Platelet- and neutrophil recovery and the ratio between CD3 and CD34+
are mainly correlated with PC2.  
The variation in donor age and survival time are poorly correlated with
both principal components.

In [ ]:
var_plot <- pca_fit |>
  tidy(matrix = "eigenvalues") |>
  mutate(Percentage = percent * 100) |>
  ggplot(mapping = aes(PC, Percentage)) +
  geom_col(alpha = 0.5, fill = "#440154FF") +
  scale_x_continuous(breaks = 1:9) +
  theme_minimal() +
  labs(y = "Percentage (%)",
       x = "Principal Component")

var_plot

The first principal component explains 31% of the variance and the
second principal component explains 15% of the variance.  
Thus, only 46% of the total variance is explained by the two principal
components.

### HLA-matching impact on survival

In [ ]:
HLA_data <- bm_data |>
  mutate(HLAmatch = fct_relevel(HLAmatch, c("7/10", "8/10", "9/10", "10/10")))

match_plot <- HLA_data |> 
  count(HLAmatch, survival_status) |> 
  group_by(HLAmatch) |> 
  mutate(survival_pct = str_c(round(n / sum(n) *100, 1), "%")) |> 
  ggplot(aes(x = HLAmatch, 
             y = n, 
             fill = survival_status)) + 
  geom_col(position = "dodge", 
           alpha = 0.5) + 
  geom_text(aes(label = survival_pct), 
            vjust = -0.5, 
            position = position_dodge(width = 0.9)) + 
  theme_minimal() + 
  scale_fill_viridis_d() + 
  labs(x = "HLA Match", 
       y = "Number of Patients", 
       fill = "Survival Status", 
       title = "Survival Status of Patients" ) + 
  scale_y_continuous(limits = c(0,55)) +
  theme(legend.position = "bottom")

match_plot 

In [ ]:
survival_plot <- ggplot(data = HLA_data, 
                        mapping = aes(x = survival_time, 
                                      y = HLAmatch, 
                                      fill = HLAmatch)) + 
  stat_density_ridges(quantile_lines = TRUE, 
                      alpha = 0.3) + 
  scale_fill_viridis_d() + 
  theme_minimal() + 
  labs(x = "Survival time (days)", 
       y= "HLA Match Grade", 
       fill = "HLA Match", 
       title = "Survival of Patients Post Stem Cell Transplantation", 
       subtitle = "Distribution of Survival Time Stratified on HLA Match Grade") + 
  theme(legend.position = "bottom")

survival_plot 

Picking joint bandwidth of 398

The PC-scores plot shows that there is no clear separation between the
samples based on the survival status and thus, the groups are not
distinguishable based on the first two principal components.

### Save plots

In [ ]:
ggsave(here("results/pca_plot.png"), width = 6, plot = pca_plot)

Saving 6 x 5 in image

Saving 6 x 5 in image

Saving 6 x 5 in image

Saving 6 x 5 in image

Saving 6 x 5 in image
Picking joint bandwidth of 398